

第1步：数据准备与知识库构建
核心技术

多格式解析

技术细节：处理PDF、HTML、Word、Markdown等格式，需提取结构化文本并保留标题、列表等逻辑结构。
工具推荐：

PyPDF2 / PDFMiner：提取PDF文本，解决扫描版PDF需配合OCR（如Tesseract）。
BeautifulSoup / Readability：清理网页HTML标签，提取正文。
Apache Tika：统一解析多种格式（包括Office文档）。




文本清洗

技术细节：去除广告、导航栏、特殊字符，标准化编码（如UTF-8），处理缩写和拼写错误。
正则表达式：匹配删除无关内容（如“点击查看详情”）。
NLP工具：Spacy的规则引擎识别保留关键实体（如产品型号、技术术语）。


元数据管理

技术细节：为文本块附加来源、创建时间、作者等信息，用于后续检索排序。
实现方案：在JSON或数据库中存储{text: "...", source: "手册_v3.pdf", page: 12}。



典型代码结构
from bs4 import BeautifulSoup
import pdfplumber

# 解析PDF
with pdfplumber.open("manual.pdf") as pdf:
    text = "\n".join([page.extract_text() for page in pdf.pages])

# 清洗网页
html = requests.get(url).text
soup = BeautifulSoup(html, "lxml")
main_text = soup.find("div", class_="main-content").get_text(strip=True)


第2步：文本切分（Chunking）
分块策略

固定窗口法：

技术细节：每块固定长度（如512 tokens），重叠窗口（overlap=10%）避免语义断裂。
代码示例：from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=64, separators=["\n\n", "\n", "。", " "]
)
chunks = splitter.split_text(text)




语义分块法：

技术细节：用NLP模型检测段落边界（如Spacy的句子分割），适合技术文档。
工具：Spacy的sentencizer组件或基于Transformer的语义分割模型。



优化技巧

动态分块：根据内容类型调整块大小（如论文摘要用256 tokens，实验细节用1024 tokens）。
分块元数据：标记块类型（标题、代码示例、表格）供检索时加权。


第3步：向量化与嵌入（Embedding）
嵌入模型选型

闭源API：

OpenAI text-embedding-3-small：1536维，适合通用场景，成本约$0.0001/1k tokens。
Cohere Embed：支持多语言，提供压缩模式（降低维度节省存储）。


开源模型：

Sentence-BERT（sentence-transformers库）：预训练模型如all-mpnet-base-v2，微调支持领域适配。
BGE（BAAI General Embedding）：中文优化，支持指令微调（在提示词前加“为这个句子生成嵌入：”）。



批量计算与加速

GPU加速：用PyTorch的DataLoader多线程批量处理。
降维：PCA或UMAP将向量从1536维降至256维，减少存储和计算量。

代码示例
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(chunks, batch_size=128, show_progress_bar=True)


第4步：向量存储与检索
数据库选型对比



数据库
适用场景
关键特性




FAISS
中小规模（百万级），本地部署
GPU加速、IVF索引、支持L2/IP相似度


Pinecone
大规模，全托管云服务
自动缩放、混合搜索（稀疏+稠密向量）


Milvus
自托管，分布式扩展
支持标量过滤、数据分片、高可用


Elasticsearch
已有ES生态集成
8.0+支持kNN搜索，可与全文检索结合



索引结构优化

HNSW（Hierarchical Navigable Small World）：

参数：efConstruction=200（构建时邻居数）、efSearch=100（搜索时扩展数），权衡精度与速度。


IVF（Inverted File Index）：

聚类为nlist=4096个分区，搜索时仅查最近簇，适合超大规模数据集。



检索逻辑
import faiss
index = faiss.IndexHNSWFlat(embedding_dim, 32)
index.add(embeddings)
distances, indices = index.search(query_embedding, k=5)


第5步：检索增强生成（RAG）
提示词工程

模板设计：基于以下上下文，精确回答问题。如果答案不在上下文中，回答“未知”。

上下文：
{context_str}

问题：{query_str}


多步推理：对复杂问题先检索多个相关块，分步生成中间结论（Chain-of-Thought）。

生成模型优化

模型选择：

GPT-4：高成本，适合对准确性要求严苛的场景（如医疗、法律）。
Llama 3 70B：开源替代，需16GB以上GPU显存，使用vLLM库加速推理。


生成参数：

temperature=0.3（降低随机性），max_tokens=500，stop_sequences=["\n参考："]。



代码流程
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=vector_db.as_retriever(),
    chain_type="stuff"  # 或"map_reduce"处理长文本
)
answer = qa_chain.run("如何拆卸联想Y7000的电池？")


第6步：评估与优化
评估指标

检索阶段：

命中率（Hit Rate）：Top-K结果中包含正确答案的比例。
MRR（Mean Reciprocal Rank）：正确答案排名的倒数均值。


生成阶段：

ROUGE-L：衡量生成文本与标准答案的词汇重叠率。
BERTScore：用BERT模型评估语义相似度。



优化手段

重排序（Rerank）：

用交叉编码器（Cross-Encoder）对Top-100检索结果重新评分。
代码示例（使用Sentence-Transformers）：from sentence_transformers import CrossEncoder
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
scores = model.predict([(query, chunk) for chunk in chunks])




混合检索：结合关键词（BM25）与语义搜索，提升召回率。


扩展：高级RAG架构

多跳检索（Multi-Hop）

场景：复杂问题需多次检索（如“Y7000的电池型号是什么？该型号的供应商有哪些？”）。
实现：用LLM生成中间问题→分步检索→综合答案。


动态数据更新

技术：监听知识库文件夹变动（如Watchdog库）→触发增量嵌入更新。




技术栈全景图
用户提问 → 解析清洗 → 分块 → 向量化 → 存储 → 检索 → 增强生成 → 评估  
                │          │           │         │           │  
                ↓          ↓           ↓         ↓           ↓  
         Apache Tika   LangChain   Sentence-BERT  FAISS    GPT-4/Llama



In [ ]:
# pip install langchain sentence-transformers faiss-cpu openai tiktoken python-dotenv

import os
from dotenv import load_dotenv
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 加载环境变量（用于OpenAI API）
load_dotenv() 

# ========================= 第1步：知识库准备 =========================
# 创建示例知识库文件（实际场景替换为你的文档）
with open("knowledge_base.md", "w", encoding="utf-8") as f:
    f.write("""
## 联想笔记本电池更换指南
1. 关机并断开所有外接电源。
2. 使用T5螺丝刀卸下底部螺丝（共8颗）。
3. 从边缘撬开底盖，注意不要损坏卡扣。
4. 找到电池连接器，轻轻拔出。
5. 更换新电池后按相反顺序组装。

## 天禧AI隐私保护功能
- 本地数据处理：用户对话内容优先在设备端处理。
- 差分隐私技术：上传数据时添加噪声保护个体信息。
- 权限控制：可随时在设置中关闭数据收集。
    """)

# ========================= 第2步：文档加载与分块 =========================
loader = TextLoader("knowledge_base.md", encoding="utf-8")
documents = loader.load()

# 智能分块（每块512字符，重叠64字符）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=64,
    separators=["\n\n", "\n", "。", " "]
)
chunks = text_splitter.split_documents(documents)

# ========================= 第3步：向量嵌入与存储 =========================
# 使用Sentence-BERT生成嵌入
embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# 创建FAISS向量库
vector_db = FAISS.from_documents(
    chunks, 
    embed_model
)
vector_db.save_local("faiss_index")  # 保存索引供后续使用

# ========================= 第4步：检索增强生成 =========================
def rag_answer(question: str) -> str:
    # 加载预存的向量库
    vector_db = FAISS.load_local(
        "faiss_index", 
        embed_model,
        allow_dangerous_deserialization=True  # FAISS需要此参数
    )
    
    # 检索最相关的3个块
    retrieved_docs = vector_db.similarity_search(question, k=3)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    # 构建提示词模板
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "基于以下上下文，用简洁的技术语言回答问题。如果无法回答，请说明未知内容。\n\n上下文：{context}"),
        ("human", "问题：{question}")
    ])
    
    # 调用生成模型（此处用OpenAI，生产环境可替换为Llama等）
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.3
    )
    chain = prompt_template | llm
    
    # 执行生成
    response = chain.invoke({"question": question, "context": context})
    return response.content

# ========================= 测试 =========================
if __name__ == "__main__":
    while True:
        question = input("\n请输入问题（输入q退出）: ")
        if question.lower() == "q":
            break
        print("\n回答：", rag_answer(question))
